In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np
import re

from StringAlphabetCreator import *
from StringLenCreator import *
from StringNumbersCharacterCreator import *
from StringSpecialCharacterCreator import *
from StringUniqueCharacterCreator import *
from CubicTransformer import *

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn import set_config
set_config(transform_output="pandas")

In [9]:
from XCopyTransformer import *
preprocessing = Pipeline(
    steps=[
        ('Preprocessing - Profiling and Datatypes', 
                ColumnTransformer(transformers=[
                    ("X", XCopyTransformer(), make_column_selector(dtype_include=None))
                ], remainder="passthrough", n_jobs=-1)
            ),
    ("Pattern calculation",     
        ColumnTransformer(transformers=[
        ("Quantity_CapitalLetters",
            Pipeline(
                steps=[
                    ("alphabet_s_big", StringAlphabetCreatorBig()),
                    ("z_transform_s_big", StandardScaler()),
                    ("cubic_transform_s_big", CubicTransformer()) 
                ]), make_column_selector(dtype_include=np.object_)),
        ("Quantity_LowerCaseLetters",
            Pipeline(
                steps=[
                    ("alphabet_s_small", StringAlphabetCreatorSmall()), 
                    ("z_transform_s_small", StandardScaler()),
                    ("cubic_transform_s_small", CubicTransformer())  
                ]), make_column_selector(dtype_include=np.object_)),
        ], remainder='passthrough', n_jobs=-1, verbose=True)
    ),

    ])

test_daten = pd.DataFrame(
    {
      "COLTestCAT1":np.array(["Katze","f788b179-4eb7-4da0-addb-8e791a00ccb9","3dfbe72a-daf9-47d2-b527-4bfe2896a695"]),  
      "COLTestCAT2":np.array(["Hund","Hund123","hund"]),  

     })

preprocessed_data = preprocessing.fit_transform(test_daten)
preprocessed_data

,Quantity_CapitalLetters__X__COLTestCAT1_LEN_ALPHABET_BIG_0,Quantity_CapitalLetters__X__COLTestCAT2_LEN_ALPHABET_BIG_1,Quantity_LowerCaseLetters__X__COLTestCAT1_LEN_ALPHABET_SMALL_0,Quantity_LowerCaseLetters__X__COLTestCAT2_LEN_ALPHABET_SMALL_1
0,2.828427,0.353553,-2.799809,-0.353553
1,-0.353553,0.353553,0.522413,-0.353553
2,-0.353553,-2.828427,0.220393,2.828427


In [11]:
preprocessing[:-1].get_feature_names_out()

array(['X__COLTestCAT1', 'X__COLTestCAT2'], dtype=object)

In [6]:
def pca_on_columns(X_filtered, number):
    """
    Hier werden die Spalten transformiert von jeder ursprünglichen Spalte.
    Um ein Ähnlichkeitsmaß zu erhalten.
    Wenn die Summe der Hauptkomponenten > 0.7 ist, dann werden diese als Spalten zurückgegeben.
    """       
    print("HIER: ", list(X_filtered.columns))
    X_col_to_list = list(X_filtered.columns)

    extract_original_col_names = re.search(r'.*__(.*?)_LEN_', X_col_to_list[0]).group(1)
    print(extract_original_col_names)

    # Initiern mit einer Komponente
    init_components = 1
    clf = PCA(n_components=init_components)
    X_pca_columns_transformed = clf.fit_transform(X_filtered)

    suffix = f"{extract_original_col_names}_components_{init_components}"

    col_names_components = []
    col_names_components.append(suffix)

    # Falls die Varianz unter 70%, dann Komponenten hinzufügen
    while np.sum(clf.explained_variance_ratio_) <= 0.7:            
        init_components += 1
        suffix = f"{extract_original_col_names}_components_{init_components}"

        # Für jede Komponente wird eine Spaltenbezeichnung hinzugefügt
        col_names_components.append(suffix)

        clf = PCA(n_components=init_components)
        # Gibt ein Numpy-Array zurück - prüfen!
        X_pca_columns_transformed = clf.fit_transform(X_filtered)
    
    # Umwandeln in einen DataFrame
    X_pca_columns_transformed.columns =  col_names_components


    return X_pca_columns_transformed

In [7]:
def regex_len_original(X):
    # Regex, um Ziffern am Ende zu finden
    pattern = re.compile(r'_\d+$')
    # Liste, um die extrahierten Ziffern zu speichern
    numbers_at_end = []

    for col in X.columns:
        match = pattern.search(col)
        if match:
            # Extrahiere nur die Ziffern (ohne den Unterstrich)
            number_at_end = match.group(0)[1:]
            numbers_at_end.append(number_at_end)

    # +1, da mit 0 angefangen wird zu zählen
    max_n = int(max(number_at_end)) + 1
    return max_n
    

def transform_columns(X):
    X_PCA_Transformed = pd.DataFrame()
    """
    Auswahl der Feature-Spalten für jede Originalspalte.
    Dazu wird erstmal ein Tensor(Array) erzeugt, welcher alle Spalten 
    für die jeweilige Originalspalte enthält.
    Der Suffix ist _LEN_....._0 , wobei 0 die entsprechende Original-Spalte zuordnet.

    """
    # Iteriert länger als nötig drüber. Es muss eigentlich pro Originalspalte nur einmal drüber iteriert werden
    # Es wird die letzte Ziffer der Spalten extrahiert, da diese ja nummeriert sind. Davon wird das Maximum genommen + 1 das ist 
    # dann die Anzahl der Originalspalten
    X_length_original = regex_len_original(X)
    # print("CHECK:",X_length_original)


    # print("CHECK LENGTH ORIG: ",X_length_original)
    for i in range(X_length_original):
        try:
            X_filtered_columns = X.filter(regex=f'_LEN_.*_{i}$')
            X_pca = pd.DataFrame()

            # Für alle Spalten der Original-Spalte iterieren...
            for col in X_filtered_columns:
                # print(X_filtered_columns)
                # Spalten filtern und zu einem Pandas DataFrame zusammenführen - passt
                X_pca = pd.concat([X_pca, X[col]], axis=1)
            # Jetzt PCA durchführen und die Spalten zurückgeben
            pca_result = pca_on_columns(X_pca, i)

            # Ergebnis der PCA zusammenführen
            X_PCA_Transformed = pd.concat([X_PCA_Transformed, pca_result], axis=1)   

        except Exception as e:
            print("Column not found")
            print(e)

    return X_PCA_Transformed

In [8]:
preprocessed_data

,Quantity_CapitalLetters__COLTestCAT1_LEN_ALPHABET_BIG_0,Quantity_CapitalLetters__COLTestCAT2_LEN_ALPHABET_BIG_1,Quantity_LowerCaseLetters__COLTestCAT1_LEN_ALPHABET_SMALL_0,Quantity_LowerCaseLetters__COLTestCAT2_LEN_ALPHABET_SMALL_1
0,2.828427,0.353553,-2.799809,-0.353553
1,-0.353553,0.353553,0.522413,-0.353553
2,-0.353553,-2.828427,0.220393,2.828427


In [9]:
def transform(X):
    print(X.columns)
    return transform_columns(X)

In [10]:
transform(preprocessed_data)

Index(['Quantity_CapitalLetters__COLTestCAT1_LEN_ALPHABET_BIG_0',
       'Quantity_CapitalLetters__COLTestCAT2_LEN_ALPHABET_BIG_1',
       'Quantity_LowerCaseLetters__COLTestCAT1_LEN_ALPHABET_SMALL_0',
       'Quantity_LowerCaseLetters__COLTestCAT2_LEN_ALPHABET_SMALL_1'],
      dtype='object')
HIER:  ['Quantity_CapitalLetters__COLTestCAT1_LEN_ALPHABET_BIG_0', 'Quantity_LowerCaseLetters__COLTestCAT1_LEN_ALPHABET_SMALL_0']
COLTestCAT1
HIER:  ['Quantity_CapitalLetters__COLTestCAT2_LEN_ALPHABET_BIG_1', 'Quantity_LowerCaseLetters__COLTestCAT2_LEN_ALPHABET_SMALL_1']
COLTestCAT2


,COLTestCAT1_components_1,COLTestCAT2_components_1
0,2.994924,-1.5
1,-1.604242,-1.5
2,-1.390682,3.0


In [11]:
preprocessed_data

,Quantity_CapitalLetters__COLTestCAT1_LEN_ALPHABET_BIG_0,Quantity_CapitalLetters__COLTestCAT2_LEN_ALPHABET_BIG_1,Quantity_LowerCaseLetters__COLTestCAT1_LEN_ALPHABET_SMALL_0,Quantity_LowerCaseLetters__COLTestCAT2_LEN_ALPHABET_SMALL_1
0,2.828427,0.353553,-2.799809,-0.353553
1,-0.353553,0.353553,0.522413,-0.353553
2,-0.353553,-2.828427,0.220393,2.828427


In [19]:
from pyod.models.pca import PCA


clf = PCA()
clf.fit(preprocessed_data)
clf.decision_scores_

array([1.06543262e+33, 7.72684300e+32, 1.05455441e+33])